# Add Abatement Values

I've split this apart from the "scrape_propertyTax" notebook just to keep things slightly more organized and shorter. But the below code could easily be added to the end of that notebook and run just fine. 

This notebook uses the found 421A properties with the tax and unit data and adds another layer of information in the tax abatement value for each building. While this was available in the PDF tax forms, the formatting made it incredibly difficult to gather. Using NYC Open Data and the Socrata API, we can easily pull tax information for each building and merge it with our already generated files off of the BBL value. 

The most up-to-date and stabel data in the abatement API is for 2018/19 so there is a slight mismatch but still highlights the level of abatement provided by this program.

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from sodapy import Socrata
from urllib.request import urlopen
from urllib.request import urlretrieve
import json
import tabula as tb
from datetime import datetime
import re
import os
from dotenv import load_dotenv

## Set Up

In [2]:
#read in property and tax data and merge into one dataframe
properties_421_2b = pd.read_csv("../data/created_data/ad36_421Properties_TaxClass2B_withTaxData.csv")
properties_421_2 = pd.read_csv("../data/created_data/ad36_421Properties_TaxClass2_withTaxData.csv")

In [3]:
combined_properties = pd.concat([properties_421_2b, properties_421_2])

In [4]:
print(properties_421_2b.shape)
print(properties_421_2.shape)
print(combined_properties.shape)

(375, 47)
(261, 47)
(636, 47)


In [5]:
combined_properties.head(2)

,Unnamed: 0,Unnamed: 0.1,the_geom,bin,cnstrct_yr,lstmoddate,lststatype,doitt_id,heightroof,feat_code,...,YEAR BUILT,BBL,Rent Stabilized,Units,Due Date,RS ID,To Drop,Total Charge,Date,Year
0,0,1,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",4594935,2009.0,2017-08-22T00:00:00.000Z,Constructed,1112303,54.768241,2100,...,2009,4005780026,Rent Stabilization Fee - Chg,0,NaN,NaN,NaN,0,2021-06-05,2021
1,1,1,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",4594935,2009.0,2017-08-22T00:00:00.000Z,Constructed,1112303,54.768241,2100,...,2009,4005780026,Rent Stabilization Fee- Chg,8,01/01/2021,43064500.0,NaN,$160.00,2020-06-06,2020


In [6]:
#Create a list of unique BBLs in the properties data. Then using that list, create a string to insert into the where statement for the NYC Open Data pull
# Where statement should look like "bble in ('########', '############', '########') and year = '2018/19'"

unique_bbl = combined_properties['BBL'].unique()

#bbl list string
bbl_list_string = "("
for i in unique_bbl:
    to_add = "'{}',".format(i)
    bbl_list_string = bbl_list_string + to_add

bbl_list_string = bbl_list_string[:-1] + ")"


#full statement 
where_statement = "bble in {} and year = '2018/19'".format(bbl_list_string)

In [7]:
# Print where statement to check values
#print(where_statement)

"bble in ('4005780026','4006600024','4008610044','4005930011','4005940055','4006220019','4005490036','4006530074','4008860030','4006530020','4007110004','4006500069','4006330051','4005970120','4008720012','4006300011','4005940029','4006330082','4005740036','4005740037','4005390007','4008920113','4006540022','4006550012','4005720030','4005770015','4005970127','4005970118','4005970119','4005970110','4005970111','4006160004','4006530090','4006530081','4006610034','4006610037','4006630010','4005680007','4006290009','4006590068','4006500081','4006520010','4006160003','4005670029','4005980023','4006530089','4008350012','4008040123','4006150023','4006250084','4005150001','4008390019','4006290010','4005520018','4006610003','4006300018','4005170021','4005690032','4005980055','4006150083','4006590077','4006290011','4006160019','4006160011','4005690027','4005510003','4005930021','4005730060','4005940034','4006520046','4005980076','4008870038','4006590078','4006600031','4006490069','4006330003','4

## Pull from NYC Open Data

This data comes from the Property Valuation and Assessment Data in NYC Open Data

I am pulling from this API: https://dev.socrata.com/foundry/data.cityofnewyork.us/yjxr-fw8i 

A quick explanation for some of the fields below: 

 - fullval:  total market value
 - avtot: assessed total value (this is what could be billed for tax purposes)
 - extot: exempted total value (this is what has been chopped off your assessed value for tax purposes. Ownwers pay on the difference between avtot and extot)


The tax rates for the classes are as follows: {1: 20.919%, 2: 12.612%}

In [8]:
load_dotenv() 

#load environment variables
API_Key = os.getenv('NYC_API_Key')
API_Secret_Key = os.getenv('NYC_API_Secret_Key')
App_token = os.getenv('NYC_App_token')

#Set up client
client = Socrata("data.cityofnewyork.us", app_token=App_token)

In [9]:
#API Docs: https://dev.socrata.com/foundry/data.cityofnewyork.us/yjxr-fw8i 
df = client.get("yjxr-fw8i", where = where_statement, limit=500)

In [10]:
df = pd.DataFrame.from_records(df)
print(df.shape)

(159, 37)


In [34]:
df[['bble', 'fullval', 'avtot', 'extot']] = df[['bble', 'fullval', 'avtot', 'extot']].astype('int')
df = df[['bble', 'boro', 'block', 'lot', 'fullval', 'avtot', 'extot', 'year', 'taxclass', 'owner', 'census_tract', 'nta', 'staddr']]

In [36]:
# add new fields to highlight actual taxes paid and exemption values
df['Taxable Value'] = df['avtot'] - df['extot']
df['Paid Taxes'] = df['Taxable Value'] * 0.12612
df['Pre_Exempt Taxes'] = df['avtot'] * 0.12612
df['Tax Loss'] = df['Pre_Exempt Taxes'] - df['Paid Taxes']

In [37]:
df

,bble,boro,block,lot,fullval,avtot,extot,year,taxclass,owner,census_tract,nta,staddr,Taxable Value,Paid Taxes,Pre_Exempt Taxes,Tax Loss
0,4005150001,4,515,1,965000,391707,369317,2018/19,2B,SINGH HEERALALL,79,Old Astoria,30-12 14 STREET,22390,2823.82680,4.940209e+04,4.657826e+04
1,4005150031,4,515,31,5304000,2386800,2115270,2018/19,2,1222 30 AVE LLC,79,Old Astoria,12-26 30 AVENUE,271530,34245.36360,3.010232e+05,2.667779e+05
2,4005170021,4,517,21,983000,399643,395323,2018/19,2B,12-07 31ST AVENUE REA,79,Old Astoria,12-07 31 AVENUE,4320,544.83840,5.040298e+04,4.985814e+04
3,4005190006,4,519,6,35200000,15840000,15229512,2018/19,2,12-15 BROADWAY ASTORI,77,Astoria,12-15 BROADWAY,610488,76994.74656,1.997741e+06,1.920746e+06
4,4005310015,4,531,15,1954063,879328,0,2018/19,2,DK 14-23 BROADWAY LLC,77,Astoria,14-23 BROADWAY,879328,110900.84736,1.109008e+05,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,4008860030,4,886,30,1272000,287721,278495,2018/19,2B,PLANA HALKIDIKIS LLC,83,Old Astoria,26-07 18 STREET,9226,1163.58312,3.628737e+04,3.512379e+04
155,4008870032,4,887,32,415000,186750,171330,2018/19,2,"HERNANDEZ, KEVIN",91,Old Astoria,25-35 18 STREET,15420,1944.77040,2.355291e+04,2.160814e+04
156,4008870035,4,887,35,3580000,1611000,1584021,2018/19,2,GILHOMES LLC,69,Old Astoria,25-27 25 ROAD,26979,3402.59148,2.031793e+05,1.997767e+05
157,4008870038,4,887,38,1798000,769311,751743,2018/19,2B,"GILHOMES,",91,Old Astoria,18-12 25 ROAD,17568,2215.67616,9.702550e+04,9.480983e+04


In [39]:
#merge with properties data
combined_properties = combined_properties.drop_duplicates(subset=("BBL"))

In [40]:
df_merge = df.merge(combined_properties, left_on='bble', right_on="BBL", how = 'inner')

In [43]:
df_merge = df_merge[['bble', 'boro', 'block', 'lot', 'fullval', 'avtot', 'extot', 'Taxable Value', 'Paid Taxes', 'Pre_Exempt Taxes', 'Tax Loss', 'Year', 'taxclass', 'owner', 'census_tract', 'nta', 'staddr', 
                        'cnstrct_yr', 'Latitude', 'Longitude', 'AssemDist', 'BOROUGH', 'NEIGHBORHOOD', 'BUILDING CLASS CATEGORY', 'BUILDING CLASS', 'RESIDENTIAL UNITS']]
df_merge = df_merge.rename(columns = {'fullval': "Market Value", "avtot": "Assessed Value Total", "extot": "Exempted Value Total", "census_tract": "Census Tract", "owner": "Owner", "nta": "NTA", "staddr": "Address",
                            "cnstrct_yr": "Construction Year", "AssemDist": "Assembly District"})

In [45]:
df_merge.head()

,bble,boro,block,lot,Market Value,Assessed Value Total,Exempted Value Total,Taxable Value,Paid Taxes,Pre_Exempt Taxes,...,Address,Construction Year,Latitude,Longitude,Assembly District,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,BUILDING CLASS,RESIDENTIAL UNITS
0,4005150001,4,515,1,965000,391707,369317,22390,2823.82680,4.940209e+04,...,30-12 14 STREET,1960.0,40.770586,-73.930419,36,4,ASTORIA,14 4-10 FAMILY RENTALS,C1,8
1,4005150031,4,515,31,5304000,2386800,2115270,271530,34245.36360,3.010232e+05,...,12-26 30 AVENUE,2008.0,40.770865,-73.930605,36,4,ASTORIA,07 RENTALS,D1,37
2,4005170021,4,517,21,983000,399643,395323,4320,544.83840,5.040298e+04,...,12-07 31 AVENUE,2004.0,40.768872,-73.933024,36,4,ASTORIA,14 4-10 FAMILY RENTALS,C1,7
3,4005190006,4,519,6,35200000,15840000,15229512,610488,76994.74656,1.997741e+06,...,12-15 BROADWAY,2011.0,40.767002,-73.934058,36,4,ASTORIA,07 RENTALS,D1,214
4,4005310015,4,531,15,1954063,879328,0,879328,110900.84736,1.109008e+05,...,14-23 BROADWAY,2018.0,40.766033,-73.932745,36,4,ASTORIA,07 RENTALS,D1,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,4008860030,4,886,30,1272000,287721,278495,9226,1163.58312,3.628737e+04,...,26-07 18 STREET,2007.0,40.773816,-73.926833,36,4,ASTORIA,14 4-10 FAMILY RENTALS,C1,7
155,4008870032,4,887,32,415000,186750,171330,15420,1944.77040,2.355291e+04,...,25-35 18 STREET,2018.0,40.774610,-73.926397,36,4,ASTORIA,07 RENTALS,C7,12
156,4008870035,4,887,35,3580000,1611000,1584021,26979,3402.59148,2.031793e+05,...,25-27 25 ROAD,2008.0,40.774799,-73.926371,36,4,ASTORIA,07 RENTALS,C1,14
157,4008870038,4,887,38,1798000,769311,751743,17568,2215.67616,9.702550e+04,...,18-12 25 ROAD,2001.0,40.774590,-73.926143,36,4,ASTORIA,14 4-10 FAMILY RENTALS,C1,10


In [46]:
df_merge.to_csv('../data/created_data/all_TaxClass2_w_Abatements.csv')